In [1]:
import sys
import os
from src.botEnv import botEnv
import filepath as file
import random 
import neat
import pybullet as p
#import visualize
#import graphviz
import time
import math
import numpy as np

In [2]:
numOfTests = 100
posFile = 'Pos3-Axis' #Auswahl an Punkten, welche getestet werden sollen
numberAxis = 3
winner_file = 'example_winner_3_axis' #welches Netz getestet werden soll

In [3]:

#Hilfsfunktionen
fitnessDevelopment = []
bestGenomes = []
topFitness = 0
collisionDevelopment = []

        
def cartToSpher(x,y,z):
    z = z-0.36
    r = math.sqrt(x*x+y*y+z*z)
    a = z/r
    theta = math.acos(a)
    phi = math.atan2(y,x)
    pol = [r,theta,phi]
    return pol

def spherToCart(rad, theta, phi):
    x = math.sin(theta) * math.cos(phi) * rad
    y = math.sin(theta) * math.sin(phi) * rad
    z = math.cos(theta) * rad + 0.36
    cart = [x,y,z]
    return cart


def calcDistance(firstPos, secondPos):
    dist = math.sqrt(((firstPos[0]-secondPos[0])**2)+((firstPos[1]-secondPos[1])**2)+((firstPos[2]-secondPos[2])**2))
    return dist

        


def getStartPos(num):
    pos = []
    rows = num/2
    for i in range(num):
        pos.append([3*(i%10), (i/10)*3, 0])
    return pos
        
        
        


In [ ]:

    
local_dir = file.getPath()
config_path = os.path.join(local_dir, 'config.txt')
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error

with open(winner_file, "rb") as f: #welches Genom geladen werden soll
    winner = pickle.load(f)
f.close()

config = neat.config.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path
)
#node_names = {-1:'Position Achse 1', -2: 'Position Achse 2', -3: 'Position Achse 4', -4: 'Distanz r', -5: 'Distanz Theta', -6: 'Distanz Phi', 0:'Achse A1', 1:'Achse A2', 2:'Achse A4'}
#node_names = {-1:'Position Achse 1', -2: 'Position Achse 2', -3: 'Position Achse 4', -4: 'Distanz X', -5: 'Distanz Y', -6: 'Distanz Z', 0:'Achse A1', 1:'Achse A2', 2:'Achse A4'}
#node_names = {-1:'Pos Achse 1', -2: 'Zielwinkel', 0:'Achse 1'}
#visualize.draw_net(config, winner, True, node_names=node_names)


collisionCount = 0
maxStep = 400
step = 0
stepCount = 0
collisionCount = 0
over = 0
under = 0
u5cm = 0
u10cm = 0
u50cm = 0
distances = []
with open(posFile, "rb") as f:
    g = pickle.load(f)
f.close()

goalKart = g
goalSpher = []
for g in goalKart:
    goalSpher.append(cartToSpher(g[0], g[1], g[2]))
    

p.connect(p.GUI)
bot = botEnv(numAxis = numberAxis)
time.sleep(1)
net = neat.nn.FeedForwardNetwork.create(winner,config)

while(step < numOfTests):
    while(stepCount<maxStep):
        if not bot.collisionFlag:
            endeffektor = bot.getBotHeadPos() #pos kartesische Koordinaten [0] = x, [1] = y, [2] = x
            endeffektorSpher = bot.getBotSpherical() #pos räumliche Polarkoordinaten [0] = r, [1] = theta, [2] = phi
            endeffektorOrientation = list(p.getEulerFromQuaternion(p.getLinkState(bot.botId,bot.numJoints-1)[1])) #[0] = roll, [1] = pitch, [2] = yaw
            
 #######################################################################################################################################################               
                #HIER DAS NETZ AUF DIE VERWENDETEN INPUTS ANPASSEN
            output = net.activate((
                                p.getJointState(bot.botId, 0)[0],
                                p.getJointState(bot.botId, 1)[0],
                                p.getJointState(bot.botId, 3)[0],
                                endeffektorSpher[0]-goalSpher[step][0],
                                endeffektorSpher[1]-goalSpher[step][1],
                                endeffektorSpher[2]-goalSpher[step][2]
                                ))
            
 #######################################################################################################################################################           
            
            if len(p.getContactPoints(bodyA=bot.botId, bodyB=bot.botId)) > 0:   #Kollision Roboter mit sich selbst
                bot.collisionFlag = True
            if len(p.getContactPoints(bodyA=bot.botId, bodyB=bot.planeId)) > 0: #Kollision Roboter Bodenplatte
                bot.collisionFlag = True
            bot.setMovement(output)
            bot.moveStep()            
        p.stepSimulation()
        time.sleep(1/240)
        stepCount += 1
    
    if not bot.collisionFlag:
        distance = calcDistance(bot.getBotHeadPos(), goalKart[step])
        distances.append(distance)
        if distance < 0.01:
            under += 1
        elif distance < 0.05:
            u5cm += 1
        elif distance < 0.1:
            u10cm += 1
        if distance > 0.1:
            over += 1
    else:
        bot = botEnv()
        collisionCount += 1
    distances.append(distance)
    print('Step: ', step, ' distance: ', distance)
    print('Pos:', goalKart[step])
    stepCount = 0
    step += 1
    print(bot.moveDir)
    
print('Points without Collision: ', step)
print('Greates Distance: ', max(distances), ' at Step: ', distances.index(max(distances)))
distanceTotal = 0
for d in distances:
    distanceTotal += d
print('Average Distance: ', distanceTotal/numOfTests)
print('Kollisionen: ', collisionCount)
print('< 1cm :', under)
print('< 5cm :', u5cm)
print('< 10cm :', u10cm)
print('< 50cm: ', u50cm)
print('> 50cm :', over)
    
    
    
    